In [80]:
import operator
import io
import os
from tqdm import  tqdm
import gensim
import jieba
import  annoy

In [ ]:
sentences = gensim.models.word2vec.Text8Corpus('fenci2.txt')
model = gensim.models.word2vec.Word2Vec(
    sentences,
    min_count=3,
    size=100
)
model.save('data/wordToVec')

In [7]:
model = gensim.models.Word2Vec.load('data/wordToVec')
wv = model.wv
wv.vector_size

word_index = annoy.AnnoyIndex(wv.vector_size,'angular')

for i, word in enumerate(tqdm(wv.index2word)):
    word_index.add_item(i,wv[word])

100%|██████████| 4718/4718 [00:00<00:00, 60020.22it/s]


In [97]:
#返回k近邻 头节点item
def get_nns(item,word_list):
    new_word=[]
    new_word_tfidf = {} #存放 词语:词频
    data_stop_path = "baidu_stopwords.txt"
    data_stop_list = open(data_stop_path, encoding='utf-8').readlines()
    data_stop_list = [line.strip("\n") for line in data_stop_list]

    # knn
    nns = word_index.get_nns_by_item(item, 10)
    for i in nns:
        if wv.index2word[i] in data_stop_list:
            continue
        new_word.append(wv.index2word[i])


    for i in new_word:
        if i in word_list.keys():
            new_word_tfidf[i] = word_list[i]
    # print(type(new_word_tfidf))
    return new_word_tfidf

In [57]:
#统计词频
def st_word():
    data_path = "fenci2.txt"
    data_stop_path = "baidu_stopwords.txt"

    data_list = open(data_path, encoding='utf-8').readlines()
    data_list = [line.strip("\n") for line in data_list]

    data_stop_list = open(data_stop_path, encoding='utf-8').readlines()
    data_stop_list = [line.strip("\n") for line in data_stop_list]
    jieba.load_userdict('bilibili.txt')
    word_list={}
    word_sum = 0
    for item in tqdm(data_list):
        seg_list = jieba.lcut(item)

        for seg_item in seg_list:
            if seg_item in data_stop_list:
                continue
            if seg_item in word_list.keys():
                word_list[seg_item] += 1
            else:
                word_list[seg_item] = 1
            word_sum+=1
    return word_list,word_sum

In [85]:
word_list,word_sum = st_word()

100%|██████████| 26887/26887 [00:05<00:00, 4677.78it/s]


In [105]:
new_word_list = {}
import random
index = random.sample(range(0,len(wv.index2word)),int(len(wv.index2word)*0.2))
for i in index:
    new_word_tfidf = get_nns(i,word_list)
    new_word_v100 = {k:v for k, v in new_word_tfidf.items() if v>=30}
    new_word_list = dict(new_word_list,**new_word_v100)
    # print(new_word_v100)

In [106]:
for k,v in new_word_list.items():
    with open('bilibili_words.txt', 'a',encoding='utf-8') as f:
        f.write(k +'\t'+str(v)+ '\n')

In [104]:
sentences